In [4]:
import pandas as pd
import numpy as np

In [18]:
energy = pd.read_excel("Lab5/En_In.xls")
gpd = pd.read_csv("Lab5/gpd.csv")
scimagojr = pd.read_excel("Lab5/scimagojr.xlsx")

In [19]:
energy = energy.drop(range(17))
energy = energy.drop(range(244, 282))
energy = energy.drop(columns=["Unnamed: 0", "Unnamed: 1"])
energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
energy

,Country,Energy Supply,Energy Supply per Capita,% Renewable
17,Afghanistan,321.0,10.0,78.669280
18,Albania,102.0,35.0,100.000000
19,Algeria,1959.0,51.0,0.551010
20,American Samoa,NaN,NaN,0.641026
21,Andorra,9.0,121.0,88.695650
...,...,...,...,...
239,Viet Nam,2554.0,28.0,45.321520
240,Wallis and Futuna Islands,0.0,26.0,0.000000
241,Yemen,344.0,13.0,0.000000
242,Zambia,400.0,26.0,99.714670


In [21]:
energy = energy.replace(["..."], np.NaN)
energy["Energy Supply"] = energy["Energy Supply"] * 1_000_000
energy

,Country,Energy Supply,Energy Supply per Capita,% Renewable
17,Afghanistan,3.210000e+14,10.0,78.669280
18,Albania,1.020000e+14,35.0,100.000000
19,Algeria,1.959000e+15,51.0,0.551010
20,American Samoa,NaN,NaN,0.641026
21,Andorra,9.000000e+12,121.0,88.695650
...,...,...,...,...
239,Viet Nam,2.554000e+15,28.0,45.321520
240,Wallis and Futuna Islands,0.000000e+00,26.0,0.000000
241,Yemen,3.440000e+14,13.0,0.000000
242,Zambia,4.000000e+14,26.0,99.714670


In [29]:
energy["Country"] = energy["Country"].replace(["Republic of Korea"], "South Korea")
energy["Country"] = energy["Country"].replace(["United States of America"], "United states")
energy["Country"] = energy["Country"].replace(["United Kingdom of Great Britain and Northern Ireland"], "United kingdom")
energy["Country"] = energy["Country"].replace(["China, Hong Kong Special Administrative Region"], "Hong Kong")

In [30]:
def rename(s):
    res = []
    for e in s:
        if e.isdigit():
            continue
        elif e == "(":
            break
        else:
            res.append(e)
    return "".join(res).rstrip()

energy["Country"] = energy["Country"].apply(lambda x: rename(x))
energy.loc[energy["Country"].isin(["American Samoa", "South Korea", "Bolivia"])]

,Country,Energy Supply,Energy Supply per Capita,% Renewable
20,American Samoa,NaN,NaN,0.641026
41,Bolivia,3.360000e+14,32.0,31.477120
181,South Korea,1.100700e+16,221.0,2.279353


In [39]:
headers = list(gpd.loc[3])
headers[0] = "Country"
for i in range(4, len(headers)):
    headers[i] = int(headers[i])
gpd.columns = headers
gpd = gpd.drop([0, 1, 2, 3])
gpd["Country"] = gpd["Country"].replace(["Korea, Rep."], "South Korea")
gpd["Country"] = gpd["Country"].replace(["Iran, Islamic Rep."], "Iran")
gpd["Country"] = gpd["Country"].replace(["Hong Kong SAR, China"], "Hong Kong")
gpd.head(1)

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
4,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN


In [40]:
df = pd.merge(scimagojr, energy, how="left", on="Country")
df = pd.merge(df, gpd, how="left", on="Country")

df = df.set_index("Country")
df = df.astype({"Rank": int})
df = df.drop(columns=[x for x in range(1960, 2006)])
df = df.drop(columns=["Country Code", "Indicator Code", "Indicator Name"])
df.columns

Index([                    'Rank',                'Documents',
              'Citable documents',                'Citations',
                 'Self-citations',   'Citations per document',
                        'H index',            'Energy Supply',
       'Energy Supply per Capita',              '% Renewable',
                             2006,                       2007,
                             2008,                       2009,
                             2010,                       2011,
                             2012,                       2013,
                             2014,                       2015],
      dtype='object')

In [41]:
df.head(3)

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+17,93.0,19.75491,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,NaN,NaN,NaN,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+16,149.0,10.23282,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12


In [44]:
df = df[:15]
df.shape

(15, 20)

In [45]:
def task8():
    years = [x for x in range(2006, 2016)]
    return df[years].mean(axis=1, skipna=True).sort_values(ascending=False)
task8()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

In [47]:
def task9():
    return "France", df.loc["France"][2015] - df.loc["France"][2006]
task9()

('France', 153345695364.24023)

In [48]:
def task10():
    res = list(df.sort_values("% Renewable", ascending=False).index.values)[0]
    return res, df.loc[res]["% Renewable"]
task10()

('Brazil', 69.64803)

In [49]:
def task11():
    df["Population"] = df["Energy Supply"] / df["Energy Supply per Capita"]
    res = list(df["Population"].sort_values(ascending=False).index.values)[4]
    return res, df.loc[res]["Population"]
task11()

('Japan', 127409395973154.36)

In [52]:
def task12():
    return df["Energy Supply per Capita"].corr(df["Citable documents"] / df["Population"], method="pearson")
task12()

0.8642899768854407

In [58]:
def task13():
    df["renew"] = df.apply(lambda x: 1 if x["% Renewable"] > df["% Renewable"].median(axis=0) else 0, axis=1)
    df.sort_values(by='Rank')
    return df["renew"]
task13()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    0
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: renew, dtype: int64

In [59]:
def task14():
    ContinentDict = {'China': 'Asia',
                     'United States': 'North America',
                     'Japan': 'Asia',
                     'United Kingdom': 'Europe',
                     'Russian Federation': 'Europe',
                     'Canada': 'North America',
                     'Germany': 'Europe',
                     'India': 'Asia',
                     'France': 'Europe',
                     'South Korea': 'Asia',
                     'Italy': 'Europe',
                     'Spain': 'Europe',
                     'Iran': 'Asia',
                     'Australia': 'Australia',
                     'Brazil': 'South America'}
    temp = pd.DataFrame(columns=['size', 'sum', 'mean', 'std'])
    df['Population'] = df['Energy Supply'] / df['Energy Supply per Capita']
    for elem, val in df.groupby(ContinentDict):
        temp.loc[elem] = [len(val), val['Population'].sum(), val['Population'].mean(), val['Population'].std()]
    temp = temp.astype({"size": int})
    return temp
task14()

,size,sum,mean,std
Asia,5,2.898666e+15,5.797333e+14,6.790979e+14
Australia,1,2.331602e+13,2.331602e+13,NaN
Europe,6,3.940587e+14,7.881174e+13,3.813228e+13
North America,2,3.523986e+13,3.523986e+13,NaN
South America,1,2.059153e+14,2.059153e+14,NaN
